In [1]:
from deepa2datasets.config import template_dir
import random
from typing import Any,List,Dict
from dataclasses import asdict

In [2]:
from deepa2datasets.nli_builder import eSNLIBuilder

In [3]:
builder = eSNLIBuilder()

In [4]:
builder.fetch_input(None)
builder._input

{'p': 'This church choir sings to the masses as they sing joyous songs from the book at a church.',
 'he': 'The church is filled with song.',
 'hn': 'The church has cracks in the ceiling.',
 'hc': 'A choir singing at a baseball game.',
 'en': ['Not all churches have cracks in the ceiling',
  'There is no indication that there are cracks in the ceiling of the church.',
  'Not all churches have cracks in the ceiling.'],
 'ee': ['"Filled with song" is a rephrasing of the "choir sings to the masses."',
  'hearing song brings joyous in the church.',
  'If the church choir sings then the church is filled with song.'],
 'ec': ['A choir sing some other songs other than book at church during the base play. they cannot see book and play base ball same time.',
  'The choir is at a chruch not a baseball game.',
  'A baseball game isn’t played at a church.']}

In [5]:
builder.configure_product()
builder._product[0]

DeepA2Item(argument_source=None, title=None, gist=None, source_paraphrase=None, context=None, argdown_reconstruction=None, erroneous_argdown=None, reason_statements=None, conclusion_statements=None, premises=None, intermediary_conclusion=None, conclusion=None, premises_formalized=None, intermediary_conclusion_formalized=None, conclusion_formalized=None, predicate_placeholders=None, entity_placeholders=None, misc_placeholders=None, distractors=None, id='123-456', metadata={'config': eSNLIConfiguration(label='entailment', argdown_template_path='esnli/argdown_generic.txt', scheme_name='modus ponens', formal_scheme=['{p}', '{p} -> {q}', '{q}'], placeholders={'p': '{premise}', 'q': '{hypothesis}'}, nl_scheme=['{{ premise | lower }}', '{{ premise | conditional(hypothesis) }}', '{{ hypothesis | lower }}']), 'argument_mask': [1, 1, 1], 'distractor_mask': [1, 1], 'label': 'entailment'})

In [6]:
builder.produce_da2item()
asdict(builder._product[0])

{'argument_source': 'the church is filled with song. "Filled with song" is a rephrasing of the "choir sings to the masses." The church has cracks in the ceiling. A choir sing some other songs other than book at church during the base play. they cannot see book and play base ball same time. this church choir sings to the masses as they sing joyous songs from the book at a church.',
 'title': None,
 'gist': None,
 'source_paraphrase': None,
 'context': None,
 'argdown_reconstruction': '(1) this church choir sings to the masses as they sing joyous songs from the book at a church.\n(2) presuming this church choir sings to the masses as they sing joyous songs from the book at a church, the church is filled with song.\n--\nwith modus ponens from (1) (2)\n--\n(3) the church is filled with song.',
 'erroneous_argdown': None,
 'reason_statements': [{'text': '"Filled with song" is a rephrasing of the "choir sings to the masses."',
   'starts_at': 32,
   'ref_reco': 2},
  {'text': 'this church ch

In [8]:
asdict(builder._product[-8])

{'argument_source': '"Filled with song" is a rephrasing of the "choir sings to the masses." this church choir sings to the masses as they sing joyous songs from the book at a church. The church has cracks in the ceiling. it is false that a choir singing at a baseball game.',
 'title': None,
 'gist': None,
 'source_paraphrase': None,
 'context': None,
 'argdown_reconstruction': '(1) this church choir sings to the masses as they sing joyous songs from the book at a church.\n(2) presuming this church choir sings to the masses as they sing joyous songs from the book at a church, it is wrong that a choir singing at a baseball game.\n--\nwith modus ponens from (1) (2)\n--\n(3) it is false that a choir singing at a baseball game.',
 'erroneous_argdown': None,
 'reason_statements': [{'text': 'this church choir sings to the masses as they sing joyous songs from the book at a church.',
   'starts_at': 71,
   'ref_reco': 1}],
 'conclusion_statements': [{'text': 'it is false that a choir singing a

In [26]:
from jinja2 import Environment, FileSystemLoader, select_autoescape
env = Environment(
    loader = FileSystemLoader(template_dir),
    autoescape=select_autoescape()
)

In [27]:
# define filters

templates_conditional = [
    "if {antecent} then {consequent}.",
    "presuming {antecent}, {consequent}.",
    "{consequent} if {antecent}.",
    "{consequent} provided {antecent}.",
    "{antecent} only if {consequent}.",
]

templates_negation = [
    "it is not the case that {sentence}.",
    "it is wrong that {sentence}.",
    "it is false that {sentence}.",
]

# strip and lower case
def lowerall(p):
    p = p.lower()
    return p

def sal(p):
    p = p.strip(" .")
    p = lowerall(p)
    return p

def negation(p):
    t = random.choice(templates_negation)
    p = t.format(sentence=sal(p))
    return p

def conditional(antecedent,consequent):
    t = random.choice(templates_conditional)
    p = t.format(antecent=sal(antecedent),consequent=sal(consequent))
    return p

In [28]:
conditional("Peter is green.", negation("The kid is swimming."))

'presuming peter is green, it is false that the kid is swimming.'

In [38]:
# register filters
env.filters['lowerall'] = lowerall
env.filters['negation'] = negation
env.filters['conditional'] = conditional

In [95]:
from dataclasses import dataclass, field

@dataclass
class eSNLIConfiguration():
    label:str
    argdown_template_path:str = "esnli/argdown_generic.txt"
    scheme_name:str = "modus ponens"
    formal_scheme:List = field(default_factory=lambda: ["{p}","{p} -> {q}", "{q}"])
    placeholders:Dict = field(default_factory=lambda: {'p':"{premise}", "q":"{hypothesis}"})
    nl_scheme:List = field(default_factory=lambda: ["{{ premise }}", "{{ premise | conditional(hypothesis) }}", "{{ hypothesis }}"]) # Jinja templates


In [97]:
conf = eSNLIConfiguration(label="entailment")
conf.formal_scheme

['{p}', '{p} -> {q}', '{q}']

In [89]:
configurations = [
    eSNLIConfiguration(
        label="entailment",
        scheme_name = "modus ponens",
        formal_scheme = ["{p}","{p} -> {q}", "{q}"],
        nl_scheme = ["{{ premise | lower }}", "{{ premise | conditional(hypothesis) }}", "{{ hypothesis | lower }}"],
        placeholders = {'p':"{premise}", "q":"{hypothesis}"},
    )
]

In [15]:
t = "hello"
t += " you"
t

'hello you'

In [90]:
data={
 "premise":"This church choir sings to the masses as they sing joyous songs from the book at a church.",
 "hypothesis":"The church is filled with song."
}
data

{'premise': 'This church choir sings to the masses as they sing joyous songs from the book at a church.',
 'hypothesis': 'The church is filled with song.'}

In [92]:
conf = configurations[0]
print({k:v.format(**data) for k,v in conf.placeholders.items()})
argument_list=[env.from_string(t).render(data) for t in conf.nl_scheme]
print(argument_list)

{'p': 'This church choir sings to the masses as they sing joyous songs from the book at a church.', 'q': 'The church is filled with song.'}
['this church choir sings to the masses as they sing joyous songs from the book at a church.', 'the church is filled with song provided this church choir sings to the masses as they sing joyous songs from the book at a church.', 'the church is filled with song.']


In [94]:
template = env.get_template("esnli/argdown_generic.txt")
print(template.render(premise1=argument_list[0],premise2=argument_list[1],conclusion=argument_list[-1],scheme=conf.scheme_name))

(1) this church choir sings to the masses as they sing joyous songs from the book at a church.
(2) the church is filled with song provided this church choir sings to the masses as they sing joyous songs from the book at a church.
--
with modus ponens from (1) (2)
--
(3) the church is filled with song.


In [52]:
print(template.render(premise="This church choir sings to the masses as they sing joyous songs from the book at a church.", hypothesis="The church is filled with song."))

(1) the church is filled with song.
(2) presuming this church choir sings to the masses as they sing joyous songs from the book at a church, it is not the case that the church is filled with song.
--
with modus tollens from (1) (2)
--
(3) it is wrong that this church choir sings to the masses as they sing joyous songs from the book at a church.
